# Problem 2 – Yield Curve Construction and Swap Valuation

Assume: 
- The year consists of 12 months each with exactly 30 days 
- ALl payments occur at the end of day on the last day of the month
- No credit risk and the the principal of all bonds is 100. 

**Today:** 2019-12-30. **Known fixings:** 3M LIBOR = 0.01570161, 6M LIBOR = 0.01980204.

Bonds (coupon paid simple at stated frequency, principal 100):
1) Price 102.33689177, quarterly 4%, maturity 2020-12-30
2) Price 104.80430234, semiannual 5%, maturity 2020-12-30
3) Price 105.1615306,  semiannual 5%, maturity 2021-06-30
4) Price 105.6581905,  quarterly 6%,  maturity 2021-06-30
5) Price 104.028999992, quarterly 5%, maturity 2021-12-30
6) Price 101.82604116,  annual 3%,   maturity 2021-12-30

We generate coupon schedules from 2019-12-30 to each maturity with end-of-month payments and 30/360 accruals. We include principal at maturity. The cashflow matrix has one column per unique payment date across all bonds; entries are the cashflows on that date (coupon + principal if maturity).

## a) Cashflow matric 

Build the **cashflow matrix** for the given fixed-rate bullet bonds (using the exact coupon schedules).


In [2]:
import numpy as np
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

#Inputs
today = date(2019,12,30)
# LIBOR fixings (simple comp, 30/360; accrual = 0.25 and 0.50)
L3M = 0.01570161
L6M = 0.01980204

# Bonds: (price, annual_coupon_rate, freq_per_year, maturity_date)
bonds = [
    (102.33689177, 0.04, 4, date(2020,12,30)),  # quarterly 4%
    (104.80430234, 0.05, 2, date(2020,12,30)),  # semiannual 5%
    (105.1615306 , 0.05, 2, date(2021, 6,30)),  # semiannual 5%
    (105.6581905 , 0.06, 4, date(2021, 6,30)),  # quarterly 6%
    (104.028999992,0.05, 4, date(2021,12,30)),  # quarterly 5%
    (101.82604116, 0.03, 1, date(2021,12,30)),  # annual 3%
]
principal = 100.0

#  Helper functions #
def eom(d):
    """Return end-of-month for the month of d."""
    first_next = date(d.year + (d.month // 12), ((d.month % 12) + 1), 1)
    return first_next - relativedelta(days=1)

def add_months_eom(d, months):
    """Add 'months' months and snap to end-of-month."""
    base = d + relativedelta(months=months)
    return eom(base)

def year_frac_30_360(d0, d1):
    """30/360 day-count fraction between d0 and d1."""
    D0 = d0.day; D1 = d1.day
    M0 = d0.month; M1 = d1.month
    Y0 = d0.year;  Y1 = d1.year
    D0 = min(D0, 30)
    D1 = 30 if D1 == 31 and D0 == 30 else min(D1, 30)
    return ((Y1 - Y0)*360 + (M1 - M0)*30 + (D1 - D0)) / 360.0

def build_schedule(today, maturity, freq):
    """Generate coupon dates from today to maturity at end-of-month with given freq/year."""
    step_months = int(12 // freq)
    dates = []
    d = today
    while True:
        d = add_months_eom(d, step_months)
        dates.append(d)
        if d >= maturity:
            break
    return dates

# ---------- Build all coupon dates ----------
all_dates = []
schedules = []
for _, c, f, mat in bonds:
    sch = build_schedule(today, mat, f)
    schedules.append(sch)
    all_dates.extend(sch)
# Unique sorted dates
all_dates = sorted(list({d for d in all_dates}))

# Map dates to time in years (30/360 from today)
T = np.array([year_frac_30_360(today, d) for d in all_dates])

# ---------- Build cashflow matrix A and price vector P ----------
A = np.zeros((len(bonds), len(all_dates)))
P = np.array([b[0] for b in bonds], dtype=float)

for j, (price, c, f, mat) in enumerate(bonds):
    sch = schedules[j]
    for d in sch:
        idx = all_dates.index(d)
        accr = 1.0/f  # simple accrual per period under 30/360 with equal spacing
        coupon_cf = principal * c * accr
        A[j, idx] += coupon_cf
        if d == mat:
            A[j, idx] += principal  # principal at maturity


In [4]:
import numpy as np
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

# ---------- Inputs ----------
today = date(2019,12,30)
principal = 100.0

# Bonds: (price, annual_coupon_rate, freq_per_year, maturity_date)
bonds = [
    (102.33689177, 0.04, 4, date(2020,12,30)),  # quarterly 4%
    (104.80430234, 0.05, 2, date(2020,12,30)),  # semiannual 5%
    (105.1615306 , 0.05, 2, date(2021, 6,30)),  # semiannual 5%
    (105.6581905 , 0.06, 4, date(2021, 6,30)),  # quarterly 6%
    (104.028999992,0.05, 4, date(2021,12,30)),  # quarterly 5%
    (101.82604116, 0.03, 1, date(2021,12,30)),  # annual 3%
]

# ---------- Helpers ----------
def eom(d):
    """End-of-month for month of d."""
    first_next = date(d.year + (d.month // 12), ((d.month % 12) + 1), 1)
    return first_next - relativedelta(days=1)

def add_months_eom(d, months):
    """Add 'months' months and snap to end-of-month."""
    base = d + relativedelta(months=months)
    return eom(base)

def year_frac_30_360(d0, d1):
    """30/360 day-count fraction between d0 and d1."""
    D0 = min(d0.day, 30)
    D1 = d1.day
    if D1 == 31 and D0 == 30:
        D1 = 30
    else:
        D1 = min(D1, 30)
    return ((d1.year - d0.year)*360 + (d1.month - d0.month)*30 + (D1 - D0)) / 360.0

def build_schedule(today, maturity, freq):
    """Coupon dates from 'today' to maturity at EOM with freq/year."""
    step_months = int(12 // freq)
    dates = []
    d = today
    while True:
        d = add_months_eom(d, step_months)
        dates.append(d)
        if d >= maturity:
            break
    return dates

# ---------- Build schedules & unique payment dates ----------
all_dates = []
schedules = []
for _, c, f, mat in bonds:
    sch = build_schedule(today, mat, f)
    schedules.append(sch)
    all_dates.extend(sch)

all_dates = sorted(list(set(all_dates)))  # unique sorted dates
T = np.array([year_frac_30_360(today, d) for d in all_dates])  # time in years

# ---------- Build cashflow matrix A and price vector P ----------
A = np.zeros((len(bonds), len(all_dates)))
P = np.array([b[0] for b in bonds], dtype=float)

for j, (price, c, f, mat) in enumerate(bonds):
    sch = schedules[j]
    accr = 1.0/f  # simple accrual per period (equal spacing, 30/360)
    coupon_cf = principal * c * accr
    for d in sch:
        k = all_dates.index(d)
        A[j, k] += coupon_cf
        if d == mat:
            A[j, k] += principal  # principal at maturity

# ---------- Display results for (a) ----------
dates_df = pd.DataFrame({"Payment date": all_dates, "T (years, 30/360)": T})
display(dates_df)

A_df = pd.DataFrame(A, columns=[d.isoformat() for d in all_dates])
A_df.index = [f"Bond {i+1}" for i in range(len(bonds))]
display(A_df.round(6))

P_df = pd.DataFrame({"Price": P}, index=A_df.index)
display(P_df)
print(f"Cashflow matrix shape A: {A.shape}  |  #bonds x #dates")


,Payment date,"T (years, 30/360)"
0,2020-03-31,0.25
1,2020-06-30,0.50
2,2020-09-30,0.75
3,2020-12-31,1.00
4,2021-03-31,1.25
5,2021-06-30,1.50
6,2021-09-30,1.75
7,2021-12-31,2.00


,2020-03-31,2020-06-30,2020-09-30,2020-12-31,2021-03-31,2021-06-30,2021-09-30,2021-12-31
Bond 1,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00
Bond 2,0.00,2.50,0.00,2.50,0.00,0.00,0.00,0.00
Bond 3,0.00,2.50,0.00,2.50,0.00,102.50,0.00,0.00
Bond 4,1.50,1.50,1.50,1.50,1.50,101.50,0.00,0.00
Bond 5,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25
Bond 6,0.00,0.00,0.00,3.00,0.00,0.00,0.00,3.00


,Price
Bond 1,102.336892
Bond 2,104.804302
Bond 3,105.161531
Bond 4,105.658191
Bond 5,104.029000
Bond 6,101.826041


Cashflow matrix shape A: (6, 8)  |  #bonds x #dates


In [ ]:
import numpy as np
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

# Inputs from the problem description
today = date(2019,12,30)
principal = 100.0
bonds = [
    (102.33689177, 0.04, 4, date(2020,12,30)),  # quarterly 4%
    (104.80430234, 0.05, 2, date(2020,12,30)),  # semiannual 5%
    (105.1615306 , 0.05, 2, date(2021, 6,30)),  # semiannual 5%
    (105.6581905 , 0.06, 4, date(2021, 6,30)),  # quarterly 6%
    (104.028999992,0.05, 4, date(2021,12,30)),  # quarterly 5%
    (101.82604116, 0.03, 1, date(2021,12,30)),  # annual 3%
]

def build_schedule(start, end, freq):
    step = 12 // freq
    dates = []
    d = start
    while True:
        d = (d + relativedelta(months=step)).replace(day=1) + relativedelta(months=1, days=-1)
        dates.append(d)
        if d >= end.replace(day=1) + relativedelta(months=1, days=-1):
            break
    return dates

# ---------- Collect all unique payment dates ----------
schedules = [build_schedule(today, mat, f) for _,_,f,mat in bonds]
all_dates = sorted(set([d for sch in schedules for d in sch]))

# ---------- Build cashflow matrix ----------
A = np.zeros((len(bonds), len(all_dates)))
for j, (_, c, f, _) in enumerate(bonds):
    sch = schedules[j]
    coupon = principal * c / f
    for k, d in enumerate(sch):
        col = all_dates.index(d)
        A[j, col] += coupon
        if d == sch[-1]:
            A[j, col] += principal

# ---------- Display ----------
A_df = pd.DataFrame(A, columns=[d.isoformat() for d in all_dates],
                    index=[f"Bond {i+1}" for i in range(len(bonds))])
display(A_df.round(2))


,2020-03-31,2020-06-30,2020-09-30,2020-12-31,2021-03-31,2021-06-30,2021-09-30,2021-12-31
Bond 1,1.00,1.00,1.00,101.00,0.00,0.00,0.00,0.00
Bond 2,0.00,2.50,0.00,102.50,0.00,0.00,0.00,0.00
Bond 3,0.00,2.50,0.00,2.50,0.00,102.50,0.00,0.00
Bond 4,1.50,1.50,1.50,1.50,1.50,101.50,0.00,0.00
Bond 5,1.25,1.25,1.25,1.25,1.25,1.25,1.25,101.25
Bond 6,0.00,0.00,0.00,3.00,0.00,0.00,0.00,103.00


## b) Find zero-coupon prices 
(Find the vector of zero coupon prices for all the times that you can based on the above information
and find the term structure of continuously compounded zero coupon spot rates (the yield curve).
Report the results and plot both curves in an appropriate diagram.)

Solve for **zero-coupon prices** \(p(0,T)\) on all reachable payment dates and compute the **continuously compounded spot rates** \(R(T) = -\ln p(0,T)/T\).


## C) 3M forward rates 

Find 3M forward rates and plot these in the diagram from b).

## d) 2 year floating rate bullet 

Find the price of a 2 year floating rate bullet note with principal 100 paying 6M LIBOR issued today. 